<a href="https://colab.research.google.com/github/prudhvirajstark/3D-point-cloud-Segmentation-and-Clustering-with-Python/blob/main/3D_point_cloud_Segmentation_and_Clustering_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install open3d

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


     |████████████████████████████████| 188.4MB 85kB/s 


In [3]:
#create paths and load data
input_path = "gdrive/My Drive/Datasets/"
dataname = "TLS_kitchen.ply"

pcd = o3d.io.read_point_cloud(input_path + dataname)
print(pcd)

In [10]:
#FIRST Segmentation Round
#Normals Computation
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6,0.6,0.6])
#o3d.visualization.draw_geometries([pcd])

PointCloud with 511026 points.

In [11]:
#3D Shape Detection with RANSAC
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])
# o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.00x + 0.00y + 1.00z + -0.96 = 0


PointCloud with 380013 points.

In [12]:
# Clustering with DBSCAN
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

# o3d.visualization.draw_geometries([pcd])

point cloud has 13 clusters


In [13]:
#RANSAC loop for Multiple planae sharpes detection

segment_models={}
segments={}
max_plane_idx=10
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

#o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [14]:
#Redefined RANSAC with Euclidean Clustering

segment_models={}
segments={}
max_plane_idx=20
rest=pcd
d_threshold=0.01
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

the best candidate is:  0
pass 1 / 20 done.
the best candidate is:  0
pass 2 / 20 done.
the best candidate is:  0
pass 3 / 20 done.
the best candidate is:  0
pass 4 / 20 done.
the best candidate is:  0
pass 5 / 20 done.
the best candidate is:  1
pass 6 / 20 done.
the best candidate is:  0
pass 7 / 20 done.
the best candidate is:  0
pass 8 / 20 done.
the best candidate is:  0
pass 9 / 20 done.
the best candidate is:  0
pass 10 / 20 done.
the best candidate is:  0
pass 11 / 20 done.
the best candidate is:  0
pass 12 / 20 done.
the best candidate is:  1
pass 13 / 20 done.
the best candidate is:  0
pass 14 / 20 done.
the best candidate is:  4
pass 15 / 20 done.
the best candidate is:  0
pass 16 / 20 done.
the best candidate is:  1
pass 17 / 20 done.
the best candidate is:  0
pass 18 / 20 done.
the best candidate is:  7
pass 19 / 20 done.
the best candidate is:  0
pass 20 / 20 done.


In [16]:
#Euclidean Clustering of the rest with DBSCan
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

# o3d.visualization.draw_geometries([segments.values()])
# o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
#o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=0.3199,front=[0.30159062875123849, 0.94077325609922868, 0.15488309545553303],lookat=[-3.9559999108314514, -0.055000066757202148, -0.27599999308586121],up=[-0.044411423633999815, -0.138726419067636, 0.98753122516983349])
# o3d.visualization.draw_geometries([rest])

point cloud has 90 clusters
